In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import time
from datetime import datetime
import scipy.spatial
import numpy as np

# Postulaciones (se_postulo = 1)

In [3]:
dtypes = pd.Series({'idaviso': np.dtype('uint32'), 'idpostulante': np.dtype('object')})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))
postulaciones = pd.read_csv('/home/lucio/Documentos/Datos/NaventDatosTP/Data/Postulaciones/Postulaciones-merge.csv',
                            dtype=column_types,parse_dates=['fechapostulacion'],infer_datetime_format=True)
postulaciones['se_postulo'] = 1
postulaciones['se_postulo'] = postulaciones[['se_postulo']].apply(pd.to_numeric,downcast='unsigned')
postulaciones = postulaciones.drop(columns=['Unnamed: 0','fechapostulacion'])

In [3]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6603752 entries, 0 to 6603751
Data columns (total 3 columns):
idaviso         uint32
idpostulante    object
se_postulo      uint8
dtypes: object(1), uint32(1), uint8(1)
memory usage: 81.9+ MB


# Vistas (se_postulo = 0)

In [4]:
dtypes = pd.Series({'idAviso': np.dtype('uint32'), 'idpostulante': np.dtype('object')})

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

vistas = pd.read_csv('/home/lucio/Documentos/Datos/NaventDatosTP/Data/Vistas/Vistas-merge.csv',dtype=column_types,parse_dates=['timestamp'],
                     infer_datetime_format=True)
vistas = vistas.drop(columns=['Unnamed: 0'])
#Considero unica vista de aviso por postulante (no me importa ni horario ni cantidad solo que lo vea)
vistas.drop(columns=['timestamp'],inplace=True)
vistas.drop_duplicates(subset=['idAviso','idpostulante'],inplace=True)
vistas['se_postulo'] = 0
vistas['se_postulo'] = vistas[['se_postulo']].apply(pd.to_numeric,downcast='unsigned')
vistas.rename(index=str,columns={'idAviso':'idaviso'},inplace=True)

In [5]:
vistas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8597915 entries, 0 to 8597914
Data columns (total 3 columns):
idaviso         uint32
idpostulante    object
se_postulo      uint8
dtypes: object(1), uint32(1), uint8(1)
memory usage: 172.2+ MB


# Outer merge

In [5]:
#Puede que haya vistas que enrealidad el postulante luego se postulo asi que verifico con el dataset merge
outer = pd.merge(postulaciones,vistas,how='outer',on=['idaviso','idpostulante'])

In [7]:
outer.isnull().sum()

idaviso               0
idpostulante          0
se_postulo_x    6823880
se_postulo_y    4829717
dtype: int64

# Agrego informacion de postulantes

In [8]:
postulantes_final = pd.read_csv('/home/fabrizio/env/NaventDatosTP/Data/POSTULANTES_FINAL.csv')
postulantes_final = postulantes_final.drop(columns=['Unnamed: 0'])
#postulantes_final

In [9]:
postulantes_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438702 entries, 0 to 438701
Data columns (total 6 columns):
idpostulante     438702 non-null object
sexo_numerico    438702 non-null int64
edad             438702 non-null int64
estado_code      438702 non-null int64
sexo_code        438702 non-null int64
nombre_code      438702 non-null int64
dtypes: int64(5), object(1)
memory usage: 20.1+ MB


In [10]:
outer = pd.merge(outer,postulantes_final,how='left',on='idpostulante')

In [11]:
outer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13427632 entries, 0 to 13427631
Data columns (total 9 columns):
idaviso          uint64
idpostulante     object
se_postulo_x     float64
se_postulo_y     float64
sexo_numerico    float64
edad             float64
estado_code      float64
sexo_code        float64
nombre_code      float64
dtypes: float64(7), object(1), uint64(1)
memory usage: 1.0+ GB


In [12]:
#Me quedo con los registros con todos los datos (por ahora desp ver si se puede usar imputing)
outer = outer[outer['nombre_code'].notnull() &
                                    outer['estado_code'].notnull() & 
                                    outer['edad'].notnull()]
outer = outer[outer['nombre_code'].notnull() &
                                    outer['estado_code'].notnull()]
outer[['edad','estado_code','sexo_code','nombre_code']] = outer[['edad','estado_code','sexo_code','nombre_code']].apply(pd.to_numeric,downcast='unsigned')
outer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10592264 entries, 0 to 13427631
Data columns (total 9 columns):
idaviso          uint64
idpostulante     object
se_postulo_x     float64
se_postulo_y     float64
sexo_numerico    float64
edad             uint16
estado_code      uint8
sexo_code        uint8
nombre_code      uint8
dtypes: float64(3), object(1), uint16(1), uint64(1), uint8(3)
memory usage: 535.4+ MB


# Agrego informacion de avisos

In [13]:
avisos = pd.read_csv('/home/fabrizio/env/NaventDatosTP/Data/optimized_avisos_detalle.csv')
avisos.drop(columns=['Unnamed: 0'],axis=1,inplace=True)

In [14]:
avisos.isnull().sum()

idaviso                      0
nombre_area_code             0
denominacion_empresa_code    0
nivel_laboral_code           0
tipo_de_trabajo_code         0
nombre_zona_code             0
dtype: int64

In [15]:
#Hago un merge de avisos y outer
outer = pd.merge(outer,avisos,how='left',on='idaviso')

In [16]:
outer[['nombre_area_code','denominacion_empresa_code','nivel_laboral_code','tipo_de_trabajo_code',
                'nombre_zona_code']] = outer[['nombre_area_code','denominacion_empresa_code','nivel_laboral_code','tipo_de_trabajo_code',
                'nombre_zona_code']].apply(pd.to_numeric,downcast='unsigned')
outer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10592264 entries, 0 to 10592263
Data columns (total 14 columns):
idaviso                      uint64
idpostulante                 object
se_postulo_x                 float64
se_postulo_y                 float64
sexo_numerico                float64
edad                         uint16
estado_code                  uint8
sexo_code                    uint8
nombre_code                  uint8
nombre_area_code             float64
denominacion_empresa_code    float64
nivel_laboral_code           float64
tipo_de_trabajo_code         float64
nombre_zona_code             float64
dtypes: float64(8), object(1), uint16(1), uint64(1), uint8(3)
memory usage: 939.4+ MB


In [18]:
outer.isnull().sum()

idaviso                            0
idpostulante                       0
se_postulo_x                 4261998
se_postulo_y                 4652238
sexo_numerico                      0
edad                               0
estado_code                        0
sexo_code                          0
nombre_code                        0
nombre_area_code             1152300
denominacion_empresa_code    1152300
nivel_laboral_code           1152300
tipo_de_trabajo_code         1152300
nombre_zona_code             1152300
dtype: int64

In [19]:
outer.drop('se_postulo_y',axis=1,inplace=True)
outer.rename(index=str,columns={'se_postulo_x':'se_postulo'},inplace=True)

In [28]:
outer['se_postulo'].fillna(0, inplace=True)

In [29]:
outer.isnull().sum()

idaviso                            0
idpostulante                       0
se_postulo                         0
sexo_numerico                      0
edad                               0
estado_code                        0
sexo_code                          0
nombre_code                        0
nombre_area_code             1152300
denominacion_empresa_code    1152300
nivel_laboral_code           1152300
tipo_de_trabajo_code         1152300
nombre_zona_code             1152300
dtype: int64

# Mas limpieza

In [30]:
limpito = outer[outer['denominacion_empresa_code'].notnull() &
                             outer['nivel_laboral_code'].notnull() &
                             outer['nombre_area_code'].notnull() &
                             outer['nombre_zona_code'].notnull() &
                             outer['tipo_de_trabajo_code'].notnull() &
                              outer['nombre_code'].notnull()]

In [31]:
limpito.isnull().sum()

idaviso                      0
idpostulante                 0
se_postulo                   0
sexo_numerico                0
edad                         0
estado_code                  0
sexo_code                    0
nombre_code                  0
nombre_area_code             0
denominacion_empresa_code    0
nivel_laboral_code           0
tipo_de_trabajo_code         0
nombre_zona_code             0
dtype: int64

In [32]:
limpito = limpito.drop_duplicates(subset=['idaviso','idpostulante'])

In [33]:
limpito.se_postulo.value_counts()

1.0    5934281
0.0    3505683
Name: se_postulo, dtype: int64

In [34]:
limpito.isnull().sum()

idaviso                      0
idpostulante                 0
se_postulo                   0
sexo_numerico                0
edad                         0
estado_code                  0
sexo_code                    0
nombre_code                  0
nombre_area_code             0
denominacion_empresa_code    0
nivel_laboral_code           0
tipo_de_trabajo_code         0
nombre_zona_code             0
dtype: int64

In [35]:
limpito.to_csv('/home/fabrizio/env/NaventDatosTP/Data/postulaciones_limpias.csv')